In [39]:
import tensorflow as tf

def setup_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        try:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
            print("config augmentation allocation mémoire gpu activée")
        except RuntimeError as e:
            print(e)

setup_gpu()

config augmentation allocation mémoire gpu activée


In [40]:
import keras
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sea
import sklearn
import scipy as sc
import nltk as nltk
import statsmodels as statsmodels
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [41]:
print("Version de TensorFlow :", tf.__version__)

Version de TensorFlow : 2.10.1


In [42]:
print("Version de Keras :", keras.__version__)

Version de Keras : 2.10.0


## Panneau de configuration

In [43]:
project_path = "C:\\Users\\charl\\Documents\\workspace\\green_ia\\cch\\"
file_nbr = '03' # numéro d'identification des csv à générer 
openfoodfact_csv_version = '01'

In [44]:
# import du csv de données openfoodfacts
openfoodfacts_csv = project_path + f"data\\openfoodfacts_{openfoodfact_csv_version}.csv"
df = pd.read_csv(openfoodfacts_csv)

C:\Users\charl\AppData\Local\Temp\ipykernel_18568\1547600537.py:3: DtypeWarning: Columns (0,11,12,14,15,16,17,23,24,25,26,27,31,32,33,34,35,36,37,44,46,47,48,50,52,56,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(openfoodfacts_csv)


# Analyse des données 

In [45]:
# affiche le nom de toutes les colonnes du df 
column_list = list(df.columns)
print(f"nbr col: {len(column_list)}, liste: {column_list}")

nbr col: 206, liste: ['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'abbreviated_product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'packaging_en', 'packaging_text', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'origins_en', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutrition_data', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group'

In [46]:
# afficher % de nan / colonne:
total_rows = df.shape[0]
nan_counts = df.isna().sum()
nan_percentage = (nan_counts / total_rows) * 100
nan_df = pd.DataFrame({'column_name': nan_percentage.index, 'percentage_nan': nan_percentage.values})

nan_percent_range = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
grouped = nan_df.groupby(pd.cut(nan_df['percentage_nan'], bins=[tranche[0] for tranche in nan_percent_range + [(100,)]]))

for tranche, group in grouped:
    print(f"range {tranche}:")
    print(group['column_name'].tolist())
    print()

range (0, 10]:
['creator', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'countries', 'countries_tags', 'countries_en', 'nutriscore_grade', 'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_en', 'ecoscore_grade', 'completeness']

range (10, 20]:
['last_image_t', 'last_image_datetime', 'image_url', 'image_small_url']

range (20, 30]:
['energy-kcal_100g', 'energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g']

range (30, 40]:
['salt_100g', 'sodium_100g']

range (40, 50]:
['brands', 'brands_tags', 'image_nutrition_url', 'image_nutrition_small_url']

range (50, 60]:
['categories', 'categories_tags', 'categories_en', 'main_category', 'main_category_en']

range (60, 70]:
['quantity', 'ingredients_analysis_tags', 'nutriscore_score', 'food_groups', 'food_groups_tags', 'food_groups_en', 'nutrient_levels_tags', 'product_quantity', 'unique_scans_n', 'popularity_tags', 'fiber_100g', 'nutrition-score-f

C:\Users\charl\AppData\Local\Temp\ipykernel_18568\3943661832.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = nan_df.groupby(pd.cut(nan_df['percentage_nan'], bins=[tranche[0] for tranche in nan_percent_range + [(100,)]]))


# Traitement des données 

In [47]:
# récupérer les colonnes intéressantes 
col_classi = [
    'pnns_groups_1',
    'pnns_groups_2',
    'energy-kcal_100g',
    'fat_100g',
    'saturated-fat_100g',
    'carbohydrates_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
    'sodium_100g',
    'brands_tags',
]
cat_df = df[col_classi] 
cat_df.tail(3)

,pnns_groups_1,pnns_groups_2,energy-kcal_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g,brands_tags
3236623,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mtr
3236624,Milk and dairy products,Dairy desserts,24.0,28.0,13.0,70.0,49.0,2.0,3.0,1.2,NaN
3236625,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edeka


In [48]:
# renommer les colonnes 
rename_col = {
    'pnns_groups_1': 'pnns1',
    'energy-kcal_100g': 'kcal', 
    'fat_100g': 'fat',
    'saturated-fat_100g': 'sat_fat',
    'carbohydrates_100g': 'carbohyd',
    'sugars_100g': 'sugar',
    'proteins_100g': 'prot',
    'salt_100g': 'salt',
    'sodium_100g': 'sodium',
    'brands_tags': 'brand',
    'pnns_groups_2': 'pnns2'
}
cat_df = cat_df.rename(columns=rename_col)
cat_df.tail(3)

,pnns1,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand
3236623,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mtr
3236624,Milk and dairy products,Dairy desserts,24.0,28.0,13.0,70.0,49.0,2.0,3.0,1.2,NaN
3236625,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edeka


In [49]:
# suppresion des lignes où pnns2 = unknow ou NaN
cat_df = cat_df.dropna(subset = ["pnns2"])
cat_df = cat_df[cat_df['pnns2'] != 'unknown']

# mélange des lignes aléatoirement 
cat_df = cat_df.sample(frac=1).reset_index(drop=True)

cat_df.tail(3)

,pnns1,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand
1163139,Sugary snacks,Sweets,591.0,42.0,6.6,43.0,38.0,8.2,0.0,0.00,dm-bio
1163140,Cereals and potatoes,Cereals,828.0,92.0,37.5,0.0,0.0,0.0,0.0,0.00,NaN
1163141,Cereals and potatoes,Bread,404.0,6.3,0.8,72.0,6.8,12.0,1.1,0.44,eliges


In [50]:
# normalisation des données numériques 
col_to_norm = ['kcal', 'fat', 'sat_fat', 'carbohyd', 'sugar', 'prot', 'salt', 'sodium']
scaler = MinMaxScaler()
cat_df[col_to_norm] = scaler.fit_transform(cat_df[col_to_norm])
cat_df.tail(3)

,pnns1,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand
1163139,Sugary snacks,Sweets,1.026678e-14,9.712684e-12,2.970000e-09,1.771555e-17,6.774648e-18,3.037116e-25,0.000000e+00,0.000000e+00,dm-bio
1163140,Cereals and potatoes,Cereals,1.438392e-14,2.127540e-11,1.687500e-08,0.000000e+00,1.737089e-19,1.530500e-25,0.000000e+00,0.000000e+00,NaN
1163141,Cereals and potatoes,Bread,7.018242e-15,1.456903e-12,3.600000e-10,2.966324e-17,1.354930e-18,3.735303e-25,1.628261e-45,1.628261e-45,eliges


# Génération df predict catégories

In [51]:
# split des df de train, test et valid
cat_train_df, cat_test_df = train_test_split(cat_df, test_size=0.1, random_state=42)
cat_train_df, cat_valid_df = train_test_split(cat_train_df, test_size=0.05, random_state=42) 

# sauvegarde au format csv 
cat_train_df.to_csv(project_path + f'\\data\\cat_train_df_{file_nbr}.csv', index=False, sep=";")
cat_test_df.to_csv(project_path + f'\\data\\cat_test_df_{file_nbr}.csv', index=False, sep=";")
cat_valid_df.to_csv(project_path + f'\\data\\cat_valid_df_{file_nbr}.csv', index=False, sep=";")